In [ ]:
sc

In [ ]:
import pandas as pd
import numpy as np
from itertools import chain

<br></br> <br></br>

In [ ]:
train_activity = pd.read_csv("original_data/train_activity.csv")
train_activity.head()

In [ ]:
train_guild = pd.read_csv("original_data/train_guild.csv")
train_guild.head()

In [ ]:
train_label = pd.read_csv("original_data/train_label.csv")
train_label.head()

In [ ]:
train_party = pd.read_csv("original_data/train_party.csv")
train_party.head()

In [ ]:
train_payment = pd.read_csv("original_data/train_payment.csv")
train_payment.head(10)

In [ ]:
train_trade = pd.read_csv("original_data/train_trade.csv")
train_trade.head()

In [ ]:
test_activity = pd.read_csv("original_data/test_activity.csv")
test_activity.head()

In [ ]:
test_guild = pd.read_csv("original_data/test_guild.csv")
test_guild.head()

In [ ]:
test_party = pd.read_csv("original_data/test_party.csv")
test_party.head()

In [ ]:
test_payment = pd.read_csv("original_data/test_payment.csv")
test_payment.head(10)

In [ ]:
test_trade = pd.read_csv("original_data/test_trade.csv")
test_trade.head()

<br></br>

In [ ]:
path = "file:///home/ubuntu/BigCon/data/"
data_set = ["train_", "test_"]
data_lst = ["activity.csv", "guild.csv", "label.csv", "party.csv", "payment.csv", "trade.csv"]

In [ ]:
def load_data_as_rdd(path=path, data_set=data_set, data_lst=data_lst) :
    rdd_lst = []
    
    for idx in data_set :
        for idx2 in data_lst :
            rdd_lst.append(sc.textFile(path+idx+idx2, 16))
            
    return rdd_lst

In [ ]:
rdd_lst = load_data_as_rdd()

<br></br><br></br>

### train data의 user id 정리

In [ ]:
train_activity_rdd = rdd_lst[0]
train_activity_id_rdd = train_activity_rdd.map(lambda line : line.split(",")[1]).filter(lambda line : line != 'acc_id')
train_activity_id_rdd = train_activity_id_rdd.distinct()

print("acc_id" in train_activity_id_rdd.collect())
train_activity_id_rdd.take(5)

In [ ]:
def guild_id(line) :
    guild_id_lst = line.split(",")[1:]
    guild_id_lst[0] = guild_id_lst[0].strip('"')
    guild_id_lst[-1] = guild_id_lst[-1].strip('"')
    
    return guild_id_lst

train_guild_rdd = rdd_lst[1]
train_guild_id_rdd = train_guild_rdd.flatMap(lambda line : guild_id(line)).filter(lambda line : line != 'guild_member_acc_id')
train_guild_id_rdd = train_guild_id_rdd.distinct()

print("guild_member_acc_id" in train_guild_id_rdd.collect())
train_guild_id_rdd.take(5)

In [ ]:
train_label_rdd = rdd_lst[2]
train_label_id_rdd = train_label_rdd.map(lambda line : line.split(",")[0]).filter(lambda line : line != 'acc_id')
train_label_id_rdd = train_label_id_rdd.distinct()

print("acc_id" in train_label_id_rdd.collect())
train_label_id_rdd.take(5)

In [ ]:
def party_id(line) :
    party_id_lst = line.split(",")[6:]
    party_id_lst[0] = party_id_lst[0].strip('"')
    party_id_lst[-1] = party_id_lst[-1].strip('"')
    
    return party_id_lst

train_party_rdd = rdd_lst[3]
train_party_id_rdd = train_party_rdd.flatMap(lambda line : party_id(line)).filter(lambda line : line != 'hashed')
train_party_id_rdd = train_party_id_rdd.distinct()

print('hashed' in train_party_id_rdd.collect())
train_party_id_rdd.take(5)

In [ ]:
train_payment_rdd = rdd_lst[4]
train_payment_id_rdd = train_payment_rdd.map(lambda line : line.split(",")[1]).filter(lambda line : line != "acc_id")
train_payment_id_rdd = train_payment_id_rdd.distinct()

print("acc_id" in train_payment_id_rdd.collect())
train_payment_id_rdd.take(5)
train_payment_id_rdd.take(5)

In [ ]:
train_trade_rdd = rdd_lst[5]
train_trade_id_rdd = train_trade_rdd.flatMap(lambda line : line.split(",")[3:5])
train_trade_id_rdd = train_trade_id_rdd.filter(lambda line : line not in ['source_acc_id','target_acc_id'])
train_trade_id_rdd = train_trade_id_rdd.distinct()

print('source_acc_id' in train_trade_id_rdd.collect())
print('target_acc_id' in train_trade_id_rdd.collect())
train_trade_id_rdd.take(5)

In [ ]:
train_user_id_lst_rdd = train_activity_id_rdd.union(train_guild_id_rdd).distinct() \
                              .union(train_label_id_rdd).distinct() \
                              .union(train_party_id_rdd).distinct() \
                              .union(train_payment_id_rdd).distinct() \
                              .union(train_trade_id_rdd).distinct()

In [ ]:
print(train_user_id_lst_rdd.count())
train_user_id_lst_rdd.take(5)

In [ ]:
train_user_id_dic = {}
train_user_id_lst = train_user_id_lst_rdd.collect()

for idx, acc_id in enumerate(train_user_id_lst) :
    train_user_id_dic[acc_id] = idx

In [ ]:
train_user_id_df = pd.DataFrame.from_records(list(train_user_id_dic.items())).rename(columns={0:"acc_id", 1:"change_id"})
print(len(train_user_id_df))
train_user_id_df.head()

<br></br><br></br>

### train data의 guild id 정리

In [ ]:
train_guild_rdd2 = rdd_lst[1]
train_guild_id_rdd2 = train_guild_rdd2.map(lambda line : line.split(",")[0]).filter(lambda line : line != 'guild_id')
train_guild_id_rdd2 = train_guild_id_rdd2.distinct()

print("guild_id" in train_guild_id_rdd2.collect())
train_guild_id_rdd2.take(5)

In [ ]:
train_guild_id_dic = {}
train_guild_id_lst = train_guild_id_rdd2.collect()

for idx, guild_id in enumerate(train_guild_id_lst) :
    train_guild_id_dic[guild_id] = idx

In [ ]:
train_guild_id_df = pd.DataFrame.from_records(list(train_guild_id_dic.items())).rename(columns={0:"guild_id", 1:"change_id"})
print(len(train_guild_id_df))
train_guild_id_df.head()

<br></br> <br></br>

### test data의 user id 정리

In [ ]:
test_activity_rdd = rdd_lst[6]
test_activity_id_rdd = test_activity_rdd.map(lambda line : line.split(",")[1]).filter(lambda line : line != 'acc_id')
test_activity_id_rdd = test_activity_id_rdd.distinct()

print("acc_id" in test_activity_id_rdd.collect())
test_activity_id_rdd.take(5)

In [ ]:
def guild_id(line) :
    guild_id_lst = line.split(",")[1:]
    guild_id_lst[0] = guild_id_lst[0].strip('"')
    guild_id_lst[-1] = guild_id_lst[-1].strip('"')
    
    return guild_id_lst

test_guild_rdd = rdd_lst[7]
test_guild_id_rdd = test_guild_rdd.flatMap(lambda line : guild_id(line)).filter(lambda line : line != 'guild_member_acc_id')
test_guild_id_rdd = test_guild_id_rdd.distinct()

print("guild_member_acc_id" in test_guild_id_rdd.collect())
test_guild_id_rdd.take(5)

In [ ]:
def party_id(line) :
    party_id_lst = line.split(",")[6:]
    party_id_lst[0] = party_id_lst[0].strip('"')
    party_id_lst[-1] = party_id_lst[-1].strip('"')
    
    return party_id_lst

test_party_rdd = rdd_lst[9]
test_party_id_rdd = test_party_rdd.flatMap(lambda line : party_id(line)).filter(lambda line : line != 'hashed')
test_party_id_rdd = test_party_id_rdd.distinct()

print('hashed' in test_party_id_rdd.collect())
test_party_id_rdd.take(5)

In [ ]:
test_payment_rdd = rdd_lst[10]
test_payment_id_rdd = test_payment_rdd.map(lambda line : line.split(",")[1]).filter(lambda line : line != "acc_id")
test_payment_id_rdd = test_payment_id_rdd.distinct()

print("acc_id" in test_payment_id_rdd.collect())
test_payment_id_rdd.take(5)
test_payment_id_rdd.take(5)

In [ ]:
test_trade_rdd = rdd_lst[11]
test_trade_id_rdd = test_trade_rdd.flatMap(lambda line : line.split(",")[3:5])
test_trade_id_rdd = test_trade_id_rdd.filter(lambda line : line not in ['source_acc_id','target_acc_id'])
test_trade_id_rdd = test_trade_id_rdd.distinct()

print('source_acc_id' in test_trade_id_rdd.collect())
print('target_acc_id' in test_trade_id_rdd.collect())
test_trade_id_rdd.take(5)

In [ ]:
test_user_id_lst_rdd = test_activity_id_rdd.union(test_guild_id_rdd).distinct() \
                              .union(test_party_id_rdd).distinct() \
                              .union(test_payment_id_rdd).distinct() \
                              .union(test_trade_id_rdd).distinct()

In [ ]:
test_user_id_dic = {}
test_user_id_lst = test_user_id_lst_rdd.collect()

for idx, acc_id in enumerate(test_user_id_lst) :
    test_user_id_dic[acc_id] = idx

In [ ]:
test_user_id_df = pd.DataFrame.from_records(list(test_user_id_dic.items())).rename(columns={0:"acc_id", 1:"change_id"})
print(len(test_user_id_df))
test_user_id_df.head()

<br></br>

### test data의 guild id 정리

In [ ]:
test_guild_rdd2 = rdd_lst[7]
test_guild_id_rdd2 = test_guild_rdd2.map(lambda line : line.split(",")[0]).filter(lambda line : line != 'guild_id')
test_guild_id_rdd2 = test_guild_id_rdd2.distinct()

print("guild_id" in test_guild_id_rdd2.collect())
test_guild_id_rdd2.take(5)

In [ ]:
test_guild_id_dic = {}
test_guild_id_lst = test_guild_id_rdd2.collect()

for idx, guild_id in enumerate(test_guild_id_lst) :
    test_guild_id_dic[guild_id] = idx

In [ ]:
test_guild_id_df = pd.DataFrame.from_records(list(test_guild_id_dic.items())).rename(columns={0:"guild_id", 1:"change_id"})
print(len(test_guild_id_df))
test_guild_id_df.head()

<br></br> <br></br>

### df 저장

In [ ]:
train_user_id_df.to_csv("./data_index/train_user_id.csv")
train_guild_id_df.to_csv("./data_index/train_guild_id.csv")
test_user_id_df.to_csv("./data_index/test_user_id.csv")
test_guild_id_df.to_csv("./data_index/test_guild_id.csv")

<br></br>

### train id 변환

In [ ]:
train_activity["acc_id"] = train_activity["acc_id"].map(lambda x : int(train_user_id_dic[x]))
train_activity.head()

In [ ]:
train_guild2 = train_guild['guild_member_acc_id'].map(lambda x : x.split(",")).apply(pd.Series)
train_guild2 = train_guild2.rename(columns = lambda x : 'member' + str(x)+'_acc_id')
train_guild3 = pd.concat([train_guild[:], train_guild2[:]], axis=1).drop("guild_member_acc_id", axis=1)

for idx in range(77) :
    col = 'member' + str(idx)+'_acc_id'
    train_guild3[col] = train_guild3[col].map(lambda x : int(train_user_id_dic[x]) if type(x)==str else x)
train_guild3["guild_id"] = train_guild3["guild_id"].map(lambda x : int(train_guild_id_dic[x]))

train_guild3.head()

In [ ]:
train_label["acc_id"] = train_label["acc_id"].map(lambda x : int(train_user_id_dic[x]))
train_label.head()

In [ ]:
print(len(train_party))
train_party.head()

In [ ]:
train_party_lst = []
length = 0

for idx in range(0, len(train_party), 100000) :
    if idx == 0 :
        pre_idx=idx
        continue
        
    temp = train_party[pre_idx:idx]
    temp2 = temp['hashed'].map(lambda x : x.split(",")).apply(pd.Series)
    temp3 = temp2.rename(columns = lambda x : 'member' + str(x)+'_acc_id')
    temp4 = pd.concat([temp[:], temp3[:]], axis=1).drop("hashed", axis=1)
    print("check", end=" ")
    
    temp_length = len(temp3.columns)
    if temp_length > length :
        length = temp_length 
        
    for idx2 in range(temp_length) :
        col = 'member' + str(idx2)+'_acc_id'
        temp4[col] = temp4[col].map(lambda x : train_user_id_dic[x] if type(x)==str else x)
        
    train_party_lst.append(temp4)
    pre_idx = idx
    print(idx)

temp = train_party['hashed'][pre_idx:]
temp2 = temp.map(lambda x : x.split(",")).apply(pd.Series)
temp3 = temp2.rename(columns = lambda x : 'member' + str(x)+'_acc_id')
temp4 = pd.concat([temp[:], temp3[:]], axis=1).drop("hashed", axis=1)
print("check", end=" ")
    
temp_length = len(temp3.columns)
if temp_length > length :
    length = temp_length 
        
for idx2 in range(temp_length) :
    col = 'member' + str(idx2)+'_acc_id'
    temp4[col] = temp4[col].map(lambda x : train_user_id_dic[x] if type(x)==str else x)
        
train_party_lst.append(temp4)
pre_idx = idx + len(temp)
print(pre_idx)
print(len(train_party))

length +=6
print(length)

for df in train_party_lst :
    if len(df.columns) < length :
        temp = length - len(df) 
        
        for idx in range(len(df)-6, len(df)-6+temp) :
            df['member' + str(idx)+'_acc_id'] = float('nan')

train_party2 = pd.concat(train_party_lst)
train_party2.head()

In [ ]:
cnt = 0
def party_id(x) :
    global cnt
    
    x += cnt
    cnt += 1
    
    return x

train_party3 = train_party2.copy()
train_party3["party_id"] = 0
train_party3["party_id"] = train_party3["party_id"].map(lambda x: party_id(x))
train_party3.head()

In [ ]:
train_payment["acc_id"] = train_payment["acc_id"].map(lambda x : int(train_user_id_dic[x]))
train_payment.head()

In [ ]:
train_trade["source_acc_id"] = train_trade["source_acc_id"].map(lambda x : int(train_user_id_dic[x]))
train_trade["target_acc_id"] = train_trade["target_acc_id"].map(lambda x : int(train_user_id_dic[x]))
train_trade.head()

<br></br> <br></br>

### test id 변환

In [ ]:
test_activity["acc_id"] = test_activity["acc_id"].map(lambda x : int(test_user_id_dic[x]))
test_activity.head()

In [ ]:
test_guild2 = test_guild['guild_member_acc_id'].map(lambda x : x.split(",")).apply(pd.Series)
test_guild2 = test_guild2.rename(columns = lambda x : 'member' + str(x)+'_acc_id')
test_guild3 = pd.concat([test_guild[:], test_guild2[:]], axis=1).drop("guild_member_acc_id", axis=1)

for idx in range(77) :
    col = 'member' + str(idx)+'_acc_id'
    test_guild3[col] = test_guild3[col].map(lambda x : int(test_user_id_dic[x]) if type(x)==str else x)
test_guild3["guild_id"] = test_guild3["guild_id"].map(lambda x : int(test_guild_id_dic[x]))

test_guild3.head()

In [ ]:
test_party_lst = []
length = 0

for idx in range(0, len(test_party), 100000) :
    if idx == 0 :
        pre_idx=idx
        continue
        
    temp = test_party[pre_idx:idx]
    temp2 = temp['hashed'].map(lambda x : x.split(",")).apply(pd.Series)
    temp3 = temp2.rename(columns = lambda x : 'member' + str(x)+'_acc_id')
    temp4 = pd.concat([temp[:], temp3[:]], axis=1).drop("hashed", axis=1)
    print("check", end=" ")
    
    temp_length = len(temp3.columns)
    if temp_length > length :
        length = temp_length 
        
    for idx2 in range(temp_length) :
        col = 'member' + str(idx2)+'_acc_id'
        temp4[col] = temp4[col].map(lambda x : test_user_id_dic[x] if type(x)==str else x)
        
    test_party_lst.append(temp4)
    pre_idx = idx
    print(idx)

temp = test_party['hashed'][pre_idx:]
temp2 = temp.map(lambda x : x.split(",")).apply(pd.Series)
temp3 = temp2.rename(columns = lambda x : 'member' + str(x)+'_acc_id')
temp4 = pd.concat([temp[:], temp3[:]], axis=1).drop("hashed", axis=1)
print("check", end=" ")
    
temp_length = len(temp3.columns)
if temp_length > length :
    length = temp_length 
        
for idx2 in range(temp_length) :
    col = 'member' + str(idx2)+'_acc_id'
    temp4[col] = temp4[col].map(lambda x : test_user_id_dic[x] if type(x)==str else x)
        
test_party_lst.append(temp4)
pre_idx = idx + len(temp)
print(pre_idx)
print(len(test_party))

length +=6
print(length)

for df in test_party_lst :
    if len(df.columns) < length :
        temp = length - len(df) 
        
        for idx in range(len(df)-6, len(df)-6+temp) :
            df['member' + str(idx)+'_acc_id'] = float('nan')

test_party2 = pd.concat(test_party_lst)
test_party2.head()

In [ ]:
cnt = 0
def party_id(x) :
    global cnt
    
    x += cnt
    cnt += 1
    
    return x

test_party3 = test_party2.copy()
test_party3["party_id"] = 0
test_party3["party_id"] = test_party3["party_id"].map(lambda x: party_id(x))
test_party3.head()

In [ ]:
test_payment["acc_id"] = test_payment["acc_id"].map(lambda x : int(test_user_id_dic[x]))
test_payment.head()

In [ ]:
test_trade["source_acc_id"] = test_trade["source_acc_id"].map(lambda x : int(test_user_id_dic[x]))
test_trade["target_acc_id"] = test_trade["target_acc_id"].map(lambda x : int(test_user_id_dic[x]))
test_trade.head()

<br></br>

### 최종 저장

In [ ]:
train_activity.to_csv("changed_data/train_activity.csv")
print("saved")
train_label.to_csv("changed_data/train_label.csv")
print("saved")
train_payment.to_csv("changed_data/train_payment.csv")
print("saved")
train_trade.to_csv("changed_data/train_trade.csv")
print("saved")

In [ ]:
test_activity.to_csv("changed_data/test_activity.csv")
print("saved")
test_payment.to_csv("changed_data/test_payment.csv")
print("saved")
test_trade.to_csv("changed_data/test_trade.csv")
print("saved")

In [ ]:
#train_party3.to_csv("changed_data/train_party.csv")
#print("saved")
#train_guild3.to_csv("changed_data/train_guild.csv")
#print("saved")

In [ ]:
#test_party3.to_csv("changed_data/test_party.csv")
#print("saved")
#test_guild3.to_csv("changed_data/test_guild.csv")
#print("saved")

<br></br> <br></br>

### guild/party csv 수정

In [ ]:
def party_id(x) :
    global cnt
    
    x += cnt
    cnt += 1
    
    return x

In [ ]:
cnt = 0

train_party4 = train_party.copy()
train_party4["party_id"] = 0
train_party4["party_id"] = train_party4["party_id"].map(lambda x : party_id(x))
train_party4["hashed"] = train_party4["hashed"].map(lambda x : [x.split(",")])

train_party5 = pd.DataFrame({"party_id": np.repeat(train_party4.party_id.values, [len(x) for x in (chain.from_iterable(train_party4.hashed))]),
                            'party_start_week': np.repeat(train_party4.party_start_week.values, [len(x) for x in (chain.from_iterable(train_party4.hashed))]),
                            'party_start_day': np.repeat(train_party4.party_start_day.values, [len(x) for x in (chain.from_iterable(train_party4.hashed))]),
                            'party_start_time': np.repeat(train_party4.party_start_time.values, [len(x) for x in (chain.from_iterable(train_party4.hashed))]),
                            'party_end_week': np.repeat(train_party4.party_end_week.values, [len(x) for x in (chain.from_iterable(train_party4.hashed))]),
                            'party_end_day': np.repeat(train_party4.party_end_day.values, [len(x) for x in (chain.from_iterable(train_party4.hashed))]),
                            'party_end_time': np.repeat(train_party4.party_end_time.values, [len(x) for x in (chain.from_iterable(train_party4.hashed))]),
                            "hashed": list(chain.from_iterable(chain.from_iterable(train_party4.hashed)))})

train_party5["hashed"] = train_party5["hashed"].map(lambda x : train_user_id_dic[x])
train_party5.head()

In [ ]:
cnt = 0

test_party4 = test_party.copy()
test_party4["party_id"] = 0
test_party4["party_id"] = test_party4["party_id"].map(lambda x : party_id(x))
test_party4["hashed"] = test_party4["hashed"].map(lambda x : [x.split(",")])

test_party5 = pd.DataFrame({"party_id": np.repeat(test_party4.party_id.values, [len(x) for x in (chain.from_iterable(test_party4.hashed))]),
                            'party_start_week': np.repeat(test_party4.party_start_week.values, [len(x) for x in (chain.from_iterable(test_party4.hashed))]),
                            'party_start_day': np.repeat(test_party4.party_start_day.values, [len(x) for x in (chain.from_iterable(test_party4.hashed))]),
                            'party_start_time': np.repeat(test_party4.party_start_time.values, [len(x) for x in (chain.from_iterable(test_party4.hashed))]),
                            'party_end_week': np.repeat(test_party4.party_end_week.values, [len(x) for x in (chain.from_iterable(test_party4.hashed))]),
                            'party_end_day': np.repeat(test_party4.party_end_day.values, [len(x) for x in (chain.from_iterable(test_party4.hashed))]),
                            'party_end_time': np.repeat(test_party4.party_end_time.values, [len(x) for x in (chain.from_iterable(test_party4.hashed))]),
                            "hashed": list(chain.from_iterable(chain.from_iterable(test_party4.hashed)))})

test_party5["hashed"] = test_party5["hashed"].map(lambda x : test_user_id_dic[x])
test_party5.head()

In [ ]:
train_party5.to_csv("changed_data/train_party.csv")
test_party5.to_csv("changed_data/test_party.csv")

<br></br><br></br>

In [ ]:
train_guild4 = train_guild.copy()
train_guild4["guild_member_acc_id"] = train_guild4["guild_member_acc_id"].map(lambda x : [x.split(",")])

train_guild5 = pd.DataFrame({"guild_id": np.repeat(train_guild4.guild_id.values, [len(x) for x in (chain.from_iterable(train_guild4.guild_member_acc_id))]),
                             "guild_member_acc_id": list(chain.from_iterable(chain.from_iterable(train_guild4.guild_member_acc_id)))})

train_guild5["guild_id"] = train_guild5["guild_id"].map(lambda x : train_guild_id_dic[x])
train_guild5["guild_member_acc_id"] = train_guild5["guild_member_acc_id"].map(lambda x : train_user_id_dic[x])
train_guild5.head()

In [ ]:
test_guild4 = test_guild.copy()
test_guild4["guild_member_acc_id"] = test_guild4["guild_member_acc_id"].map(lambda x : [x.split(",")])

test_guild5 = pd.DataFrame({"guild_id": np.repeat(test_guild4.guild_id.values, [len(x) for x in (chain.from_iterable(test_guild4.guild_member_acc_id))]),
                             "guild_member_acc_id": list(chain.from_iterable(chain.from_iterable(test_guild4.guild_member_acc_id)))})

test_guild5["guild_id"] = test_guild5["guild_id"].map(lambda x : test_guild_id_dic[x])
test_guild5["guild_member_acc_id"] = test_guild5["guild_member_acc_id"].map(lambda x : test_user_id_dic[x])
test_guild5.head()

In [ ]:
train_guild5.to_csv("changed_data/train_guild.csv")
test_guild5.to_csv("changed_data/test_guild.csv")

<br></br> <br></br>

### 확인

In [ ]:
def check(data) :
    for d in list(set(list(data))) :
        if type(d) == str :
            print(d)

In [ ]:
check(train_activity.acc_id.values)
check(train_label.acc_id.values)
check(train_payment.acc_id.values)
check(train_trade.source_acc_id.values)
check(train_trade.target_acc_id.values)

In [ ]:
check(test_activity.acc_id.values)
check(test_payment.acc_id.values)
check(test_trade.source_acc_id.values)
check(test_trade.target_acc_id.values)

In [ ]:
check(train_party5.hashed.values)
check(test_party5.hashed.values)

In [ ]:
check(train_guild5.guild_member_acc_id.values)
check(test_guild5.guild_member_acc_id.values)

In [ ]:
check(train_party5.party_id.values)
check(test_party5.party_id.values)

In [ ]:
check(train_guild5.guild_id.values)
check(test_guild5.guild_id.values)

<br></br><br></br>

### payment 수정

In [ ]:
train_payment = pd.read_csv("original_data/train_payment.csv")
test_payment = pd.read_csv("original_data/test_payment.csv")

In [ ]:
train_user_id_df = pd.read_csv("data_index/train_user_id.csv").drop("Unnamed: 0" ,axis=1).set_index("acc_id")
test_user_id_df = pd.read_csv("data_index/test_user_id.csv").drop("Unnamed: 0" ,axis=1).set_index("acc_id")

In [ ]:
train_user_id_dic = train_user_id_df.to_dict()["change_id"]
test_user_id_dic = test_user_id_df.to_dict()["change_id"]

In [ ]:
train_payment.acc_id = train_payment.acc_id.map(lambda x : int(train_user_id_dic[x]))
test_payment.acc_id = test_payment.acc_id.map(lambda x : int(test_user_id_dic[x]))

In [ ]:
train_payment.head()

In [ ]:
test_payment.head()

In [ ]:
train_payment.to_csv("changed_data/train_payment.csv")
test_payment.to_csv("changed_data/test_payment.csv")

In [ ]:
train_payment2 = train_payment.copy()
train_payment2 = train_payment.set_index("acc_id")

train_payment2.head()

In [ ]:
train_payment2 = train_payment2.pivot(index=train_payment2.index, columns='payment_week').xs('payment_amount', axis=1, drop_level=True)
train_payment2.head()

In [ ]:
test_payment2 = test_payment.copy()
test_payment2 = test_payment.set_index("acc_id")

test_payment2.head()

In [ ]:
test_payment2 = test_payment2.pivot(index=test_payment2.index, columns='payment_week').xs('payment_amount', axis=1, drop_level=True)
test_payment2.head()

In [ ]:
train_payment2.to_csv("changed_data/train_payment2.csv")
test_payment2.to_csv("changed_data/test_payment2.csv")